In [1]:
import h5py
import keras
import numpy as np
import json
import os
import uuid
import yaml

from attlayer import AttentionWeightedAverage
#from avglayer import MaskAverage
from copy import deepcopy
#from finetuning import (sampling_generator, finetuning_callbacks)
from operator import itemgetter
#from global_variables import NB_TOKENS, NB_EMOJI_CLASSES
from keras.layers import *
from keras.layers.merge import concatenate
from keras.layers import Input, Bidirectional, Embedding, Dense, Dropout, SpatialDropout1D, LSTM, Activation
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.regularizers import L1L2 
from pathlib import Path
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score
from os.path import exists

Using TensorFlow backend.


In [2]:
def elsa_architecture(nb_classes, nb_tokens, maxlen, feature_output=False, embed_dropout_rate=0, final_dropout_rate=0, embed_dim=300,
                    embed_l2=1E-6, return_attention=False, load_embedding=False, pre_embedding=None, high=False, LSTM_hidden=512, LSTM_drop=0.5):
    """
    Returns the DeepMoji architecture uninitialized and
    without using the pretrained model weights.
    # Arguments:
        nb_classes: Number of classes in the dataset.
        nb_tokens: Number of tokens in the dataset (i.e. vocabulary size).
        maxlen: Maximum length of a token.
        feature_output: If True the model returns the penultimate
                        feature vector rather than Softmax probabilities
                        (defaults to False).
        embed_dropout_rate: Dropout rate for the embedding layer.
        final_dropout_rate: Dropout rate for the final Softmax layer.
        embed_l2: L2 regularization for the embedding layerl.
        high: use or not the highway network
    # Returns:
        Model with the given parameters.
    """
    class NonMasking(Layer):   
        def __init__(self, **kwargs):   
            self.supports_masking = True  
            super(NonMasking, self).__init__(**kwargs)   

        def build(self, input_shape):   
            input_shape = input_shape   

        def compute_mask(self, input, input_mask=None):   
            # do not pass the mask to the next layers   
            return None   

        def call(self, x, mask=None):   
            return x   

        def get_output_shape_for(self, input_shape):   
            return input_shape 
    # define embedding layer that turns word tokens into vectors
    # an activation function is used to bound the values of the embedding
    model_input = Input(shape=(maxlen,), dtype='int32')
    embed_reg = L1L2(l2=embed_l2) if embed_l2 != 0 else None

    if not load_embedding and pre_embedding is None:
        embed = Embedding(input_dim=nb_tokens, output_dim=embed_dim, mask_zero=True,input_length=maxlen,embeddings_regularizer=embed_reg,
                          name='embedding')
    else:
        embed = Embedding(input_dim=nb_tokens, output_dim=embed_dim, mask_zero=True,input_length=maxlen, weights=[pre_embedding],
                          embeddings_regularizer=embed_reg,trainable=True, name='embedding')
    if high:
        x = NonMasking()(embed(model_input))
    else:
        x = embed(model_input)
    x = Activation('tanh')(x)

    # entire embedding channels are dropped out instead of the
    # normal Keras embedding dropout, which drops all channels for entire words
    # many of the datasets contain so few words that losing one or more words can alter the emotions completely
    if embed_dropout_rate != 0:
        embed_drop = SpatialDropout1D(embed_dropout_rate, name='embed_drop')
        x = embed_drop(x)

    # skip-connection from embedding to output eases gradient-flow and allows access to lower-level features
    # ordering of the way the merge is done is important for consistency with the pretrained model
    lstm_0_output = Bidirectional(LSTM(LSTM_hidden, return_sequences=True, dropout=LSTM_drop), name="bi_lstm_0" )(x)
    lstm_1_output = Bidirectional(LSTM(LSTM_hidden, return_sequences=True, dropout=LSTM_drop), name="bi_lstm_1" )(lstm_0_output)
    x = concatenate([lstm_1_output, lstm_0_output, x])
    if high:
        x = TimeDistributed(Highway(activation='tanh', name="high"))(x)
    # if return_attention is True in AttentionWeightedAverage, an additional tensor
    # representing the weight at each timestep is returned
    weights = None
    x = AttentionWeightedAverage(name='attlayer', return_attention=return_attention)(x)
    #x = MaskAverage(name='attlayer', return_attention=return_attention)(x)
    if return_attention:
        x, weights = x

    if not feature_output:
        # output class probabilities
        if final_dropout_rate != 0:
            x = Dropout(final_dropout_rate)(x)

        if nb_classes > 2:
            outputs = [Dense(nb_classes, activation='softmax', name='softmax')(x)]
        else:
            outputs = [Dense(1, activation='sigmoid', name='softmax')(x)]
    else:
        # output penultimate feature vector
        outputs = [x]

    if return_attention:
        # add the attention weights to the outputs if required
        outputs.append(weights)

    return Model(inputs=[model_input], outputs=outputs)

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
cur_lan = "elsa_es"
maxlen = 30
batch_size = 250
lr = 3e-4
epoch_size = 25000
nb_epochs = 1000
patience = 5
checkpoint_weight_path = "./ckpt"
loss = "categorical_crossentropy"
optim = "adam"
vocab_path = "/data/elsa"
nb_classes=64

In [4]:
LSTM_hidden = 512
LSTM_drop = 0.5
final_dropout_rate = 0.5
embed_dropout_rate = 0.0
high = False
load_embedding = True
embed_dim = 200

In [5]:
steps = int(epoch_size/batch_size)

wv_path = Path(vocab_path).joinpath("{:s}_wv.npy".format(cur_lan)).as_posix()
X_path = Path(vocab_path).joinpath("{:s}_X.npy".format(cur_lan)).as_posix()
y_path = Path(vocab_path).joinpath("{:s}_y.npy".format(cur_lan)).as_posix()

word_vec = np.load(wv_path, allow_pickle=True)
input_vec, input_label = np.load(X_path, allow_pickle=True), np.load(y_path, allow_pickle=True)
nb_tokens, input_len = len(word_vec), len(input_label)

#please modify the checkpoint_weight_path
#checkpoint_weight_path = '/storage1/user/ss/tmoji_ori/weight/tmoji-lstm-checkpoint-%s-h-1.hdf5' % cur_lan

#idx_shuffle = list(range(input_len))
#np.random.shuffle(idx_shuffle)
#idx_train, idx_val, idx_test = idx_shuffle[ :int(input_len*0.7) ], idx_shuffle[int(input_len*0.7):int(input_len*0.9)], idx_shuffle[int(input_len*0.9):]

train_end = int(input_len*0.7)
val_end = int(input_len*0.9)

(X_train, y_train) = (input_vec[:train_end], input_label[:train_end])
(X_val, y_val) = (input_vec[train_end:val_end], input_label[train_end:val_end])
(X_test, y_test) = (input_vec[val_end:], input_label[val_end:])

In [6]:
model = elsa_architecture(nb_classes=nb_classes, nb_tokens=nb_tokens, maxlen=maxlen, final_dropout_rate=final_dropout_rate, embed_dropout_rate=embed_dropout_rate, 
                          load_embedding=True, pre_embedding=word_vec, high=high, embed_dim=embed_dim)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 200)      21534400    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 30, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectio

In [7]:
if optim == 'adam':
    adam = Adam(clipnorm=1, lr=lr)
    model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
elif optim == 'rmsprop':
    model.compile(loss=loss, optimizer='rmsprop', metrics=['accuracy'])

model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=nb_epochs,
          validation_data=(X_val, y_val),
          callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto')],
          verbose=True)

#callbacks = finetuning_callbacks(checkpoint_weight_path, patience, verbose=1)
#for i in range(2):
    #train_gen = sampling_generator(X_train, y_train, batch_size, upsample=False, epoch_size=epoch_size)
    #model.fit_generator(train_gen, steps_per_epoch=steps, epochs=nb_epochs,validation_data=(X_val, y_val),validation_steps=steps, callbacks=callbacks, verbose=True)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 160507 samples, validate on 45860 samples
Epoch 1/1000
160507/160507 [==============================] - 198s 1ms/step - loss: 3.3931 - acc: 0.1840 - val_loss: 2.9536 - val_acc: 0.2618
Epoch 2/1000
160507/160507 [==============================] - 185s 1ms/step - loss: 2.8356 - acc: 0.2866 - val_loss: 2.7162 - val_acc: 0.3152
Epoch 3/1000
160507/160507 [==============================] - 187s 1ms/step - loss: 2.5833 - acc: 0.3436 - val_loss: 2.6072 - val_acc: 0.3474
Epoch 4/1000
160507/160507 [==============================] - 186s 1ms/step - loss: 2.3885 - acc: 0.3887 - val_loss: 2.5538 - val_acc: 0.3660
Epoch 5/1000
160507/160507 [==============================] - 185s 1ms/step - loss: 2.2192 - acc: 0.4280 - val_loss: 2.5381 - val_acc: 0.3746
Epoch 6/1000
160507/160507 [==============================] - 183s 1ms/step - loss: 2.0582 - acc: 0.4671 - val_lo

In [8]:
_, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

print(acc)

0.38744003543726907


In [9]:
token2index = json.loads(open("/data/elsa/elsa_es_vocab.txt", "r").read())

freq = {line.split()[0]: int(line.split()[1]) for line in open("/data/elsa/elsa_es_emoji.txt").readlines()}
freq_topn = sorted(freq.items(), key=itemgetter(1), reverse=True)[:nb_classes]

In [10]:
y_pred = model.predict(X_test)

In [11]:
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=[e[0] for e in freq_topn]))

              precision    recall  f1-score   support

           😂       0.34      0.51      0.41      2181
           ❤       0.31      0.45      0.37      1506
           😍       0.30      0.41      0.35      1454
           🤣       0.24      0.25      0.24       754
           😭       0.33      0.39      0.35       951
           👏       0.51      0.53      0.52       575
           🔥       0.50      0.49      0.50       617
           💜       0.39      0.37      0.38       409
           😘       0.24      0.31      0.27       372
           ♀       0.64      0.86      0.73       197
           👇       0.56      0.49      0.52       422
           😈       0.45      0.46      0.46       365
           🔴       0.60      0.66      0.63       477
           🙏       0.52      0.57      0.54       433
           🤦       0.60      0.75      0.67       469
           💪       0.37      0.40      0.38       374
           🤔       0.36      0.37      0.36       501
           💙       0.36    